**Chapter 4: Building Capable Assistants**

In [1]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

from config import set_environment
set_environment()

## Mitigating hallucinations through fact-checking

In [ ]:
from langchain.chains import LLMCheckerChain
from langchain.llms import OpenAI

llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0.7
    )
text = "What type of mammal lays the biggest eggs?"
checker_chain = LLMCheckerChain.from_llm(llm, verbose=True)
checker_chain.run(text)

## Summarizing information

### Basic prompting

In [ ]:
from langchain import OpenAI

prompt = """
Summarize this text in one sentence:

{text}
"""

llm = OpenAI(model="gpt-3.5-turbo-instruct")
text = "let me tell you a boring story from when I was young..."

summary = llm(prompt.format(text=text))
summary

In [ ]:
from langchain_decorators import llm_prompt

@llm_prompt(llm=OpenAI(model="gpt-3.5-turbo-instruct"))
def summarize(text:str, length="short") -> str:
    """
    Summarize this text in {length} length:

    {text}
    """
    return

summary = summarize(text="let me tell you a boring story from when I was young...")
summary

### Prompt templates

In [ ]:
from langchain import PromptTemplate, OpenAI
from langchain.schema import StrOutputParser

llm = OpenAI(model="gpt-3.5-turbo-instruct")
prompt = PromptTemplate.from_template(
    "Summarize this text: {text}?"
)

runnable = prompt | llm | StrOutputParser()
summary = runnable.invoke({"text": text})
summary

### Chain of density

In [ ]:
template = """Article: {text}
You will generate increasingly concise, entity-dense summaries of the
above article.
Repeat the following 2 steps 5 times.
Step 1. Identify 1-3 informative entities (";" delimited) from the article
which are missing from the previously generated summary.
Step 2. Write a new, denser summary of identical length which covers every
entity and detail from the previous summary plus the missing entities.
A missing entity is:
- relevant to the main story,
- specific yet concise (5 words or fewer),
- novel (not in the previous summary),
- faithful (present in the article),
- anywhere (can be located anywhere in the article).
Guidelines:
- The first summary should be long (4-5 sentences, ~80 words) yet highly
non-specific, containing little information beyond the entities marked
as missing. Use overly verbose language and fillers (e.g., "this article
discusses") to reach ~80 words.
- Make every word count: rewrite the previous summary to improve flow and
make space for additional entities.
- Make space with fusion, compression, and removal of uninformative
phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained,
i.e., easily understood without the article.
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made,
add fewer new entities.
Remember, use the exact same number of words for each summary.
Answer in JSON. The JSON should be a list (length 5) of dictionaries whose
keys are "Missing_Entities" and "Denser_Summary".
"""

In [ ]:
genai_history = """The academic discipline of artificial intelligence was established at a research workshop held at Dartmouth College in 1956 and has experienced several waves of advancement and optimism in the decades since.[20] Since its inception, researchers in the field have raised philosophical and ethical arguments about the nature of the human mind and the consequences of creating artificial beings with human-like intelligence; these issues have previously been explored by myth, fiction and philosophy since antiquity.[21] The concept of automated art dates back at least to the automata of ancient Greek civilization, where inventors such as Daedalus and Hero of Alexandria were described as having designed machines capable of writing text, generating sounds, and playing music.[22][23] The tradition of creative automatons has flourished throughout history, exemplified by Maillardet's automaton created in the early 1800s.[24]
Artificial Intelligence is an idea that has been captivating society since the mid-20th century. It began with science fiction familiarizing the world with the concept but the idea wasn't fully seen in the scientific manner until Alan Turing, a polymath, was curious about the feasibility of the concept. Turing's groundbreaking 1950 paper, "Computing Machinery and Intelligence," posed fundamental questions about machine reasoning similar to human intelligence, significantly contributing to the conceptual groundwork of AI. The development of AI was not very rapid at first because of the high costs and the fact that computers were not able to store commands. This changed during the 1956 Dartmouth Summer Research Project on AI where there was an inspiring call for AI research, setting the precedent for two decades of rapid advancements in the field.[25]
Since the founding of AI in the 1950s, artists and researchers have used artificial intelligence to create artistic works. By the early 1970s, Harold Cohen was creating and exhibiting generative AI works created by AARON, the computer program Cohen created to generate paintings.[26]
Markov chains have long been used to model natural languages since their development by Russian mathematician Andrey Markov in the early 20th century. Markov published his first paper on the topic in 1906,[27][28][29] and analyzed the pattern of vowels and consonants in the novel Eugeny Onegin using Markov chains. Once a Markov chain is learned on a text corpus, it can then be used as a probabilistic text generator.[30][31]
The field of machine learning often uses statistical models, including generative models, to model and predict data. Beginning in the late 2000s, the emergence of deep learning drove progress and research in image classification, speech recognition, natural language processing and other tasks. Neural networks in this era were typically trained as discriminative models, due to the difficulty of generative modeling.[32]
In 2014, advancements such as the variational autoencoder and generative adversarial network produced the first practical deep neural networks capable of learning generative models, as opposed to discriminative ones, for complex data such as images. These deep generative models were the first to output not only class labels for images but also entire images.
In 2017, the Transformer network enabled advancements in generative models compared to older Long-Short Term Memory models,[33] leading to the first generative pre-trained transformer (GPT), known as GPT-1, in 2018.[34] This was followed in 2019 by GPT-2 which demonstrated the ability to generalize unsupervised to many different tasks as a Foundation model.[35]
In 2021, the release of DALL-E, a transformer-based pixel generative model, followed by Midjourney and Stable Diffusion marked the emergence of practical high-quality artificial intelligence art from natural language prompts.
In March 2023, GPT-4 was released. A team from Microsoft Research argued that "it could reasonably be viewed as an early (yet still incomplete) version of an artificial general intelligence (AGI) system".[36] Other scholars have disputed that GPT-4 reaches this threshold, calling generative AI "still far from reaching the benchmark of ‘general human intelligence’" as of 2023.[37] In 2023, Meta released an AI model called ImageBind which combines data from text, images, video, thermal data, 3D data, audio, and motion which is expected to allow for more immersive generative AI content.[38][39]
"""

In [ ]:
#TODO: output in JSON format
from langchain import PromptTemplate, OpenAI
from langchain_core.output_parsers import JsonOutputParser

llm = OpenAI(model="gpt-3.5-turbo-instruct")
prompt = PromptTemplate.from_template(template=template)

runnable = prompt | llm
summary = runnable.invoke({"text": genai_history})
summary

### Map-Reduce pipelines

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain import OpenAI
from langchain.document_loaders import PyPDFLoader

In [ ]:
pdf_file_path = "data/databricks_vs_snowflake.pdf"
pdf_loader = PyPDFLoader(pdf_file_path)
docs = pdf_loader.load_and_split()
llm = OpenAI(model="gpt-3.5-turbo-instruct")
chain = load_summarize_chain(llm, chain_type="map_reduce")
chain.run(docs)

**Test `summarize` package**

In [ ]:
from summarize import create_pdf_summary

In [ ]:
create_pdf_summary('data/attention_is_all_you_need.pdf')

### Monitoring token usage

In [ ]:
from langchain import OpenAI, PromptTemplate
from langchain.callbacks import get_openai_callback

llm_chain = PromptTemplate.from_template("Tell me a joke about {topic}!") | OpenAI(model="gpt-3.5-turbo-instruct")
with get_openai_callback() as cb:
    response = llm_chain.invoke(dict(topic="light bulbs"))
    print(response)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

In [ ]:
input_list = [
    {"product": "socks"},
    {"product": "computer"},
    {"product": "shoes"}
]
llm_chain.generate(input_list)

## Extracting information from documents

In [ ]:
from typing import Optional
from pydantic import BaseModel

class Experience(BaseModel):
    start_date: Optional[str]
    end_date: Optional[str]
    description: Optional[str]
    
class Study(Experience):
    degree: Optional[str]
    university: Optional[str]
    country: Optional[str]
    grade: Optional[str]
    
class WorkExperience(Experience):
    company: str
    job_title: str
    
class Resume(BaseModel):
    first_name: str
    last_name: str
    linkedin_url: Optional[str]
    email_address: Optional[str]
    nationality: Optional[str]
    skill: Optional[str]
    study: Optional[Study]
    work_experience: Optional[WorkExperience]
    hobby: Optional[str]

In [ ]:
from langchain.chains import create_extraction_chain_pydantic
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader

In [ ]:
pdf_file_path = "data/openresume-resume.pdf"
pdf_loader = PyPDFLoader(pdf_file_path)
docs = pdf_loader.load_and_split()
# please note that function calling is not enabled for all models!
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0613")
chain = create_extraction_chain_pydantic(pydantic_schema=Resume, llm=llm)
chain.run(docs)

**Test `information_extraction` package**

In [ ]:
import information_extraction

information_extraction.parse_cv("data/openresume-resume.pdf")

## Answering questions with tools

### Information retrieval with tools

In [ ]:
from langchain.agents import (
    AgentExecutor, AgentType, initialize_agent, load_tools
)
from langchain.chat_models import ChatOpenAI

def load_agent() -> AgentExecutor:
    llm = ChatOpenAI(temperature=0, streaming=True)
    tools = load_tools(
        tool_names=["ddg-search", "wolfram-alpha", "arxiv", "wikipedia"],
        llm=llm
    )
    return initialize_agent(
        tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
    )

### Building a visual interface

In [ ]:
import streamlit as st
from langchain.callbacks import StreamlitCallbackHandler

chain = load_agent()
st_callback = StreamlitCallbackHandler(st.container())

if prompt := st.chat_input():
    st.chat_message("user").write(prompt)
    with st.chat_message("assistant"):
        st_callback = StreamlitCallbackHandler(st.container())
        response = chain.run(prompt, callbacks=[st_callback])
        st.write(response)

## Exploring reasoning strategies

In [2]:
from typing import Literal
from langchain.agents import initialize_agent, load_tools, AgentType
from langchain.chains.base import Chain
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import (
    load_chat_planner, load_agent_executor, PlanAndExecute
)

In [3]:
ReasoningStrategies = Literal["zero-shot-react", "plan-and-solve"]

def load_agent(
    tool_names: list[str],
    strategy: ReasoningStrategies = "zero-shot-react"
) -> Chain:
    llm = ChatOpenAI(temperature=0, streaming=True)
    tools = load_tools(
        tool_names=tool_names,
        llm=llm
    )
    if strategy == "plan-and-solve":
        planner = load_chat_planner(llm)
        executor = load_agent_executor(llm, tools, verbose=True)
        return PlanAndExecute(planner=planner, executor=executor, verbose=True)
        
    return initialize_agent(
        tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
    )

In [4]:
tool_names = ["ddg-search", "wolfram-alpha", "wikipedia"]

In [5]:
plan_chain = load_agent(tool_names=tool_names, strategy="plan-and-solve")
action_chain = load_agent(tool_names=tool_names, strategy="zero-shot-react")
question = "What is a plan-and-solve agent in the context of LLM?"

In [6]:
plan_chain.run(question)



> Entering new PlanAndExecute chain...
steps=[Step(value='Define what a plan-and-solve agent is in the context of LLM.'), Step(value='Explain how a plan-and-solve agent operates within the framework of LLM.'), Step(value='Provide examples or use cases to illustrate the concept of a plan-and-solve agent in LLM.'), Step(value="Summarize the key characteristics and benefits of using a plan-and-solve agent in LLM.\nGiven the above steps taken, please respond to the user's original question.\n")]

> Entering new AgentExecutor chain...
I will need to search for information on what a plan-and-solve agent is in the context of LLM to provide an accurate definition.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Plan-and-solve agent in the context of LLM"
}
```
Observation: Page: Large language model
Summary: A large language model (LLM) is a language model notable for its ability to achieve general-purpose language generation and other natural language processing tasks such as clas

'A plan-and-solve agent in the context of Large Language Models (LLMs) is designed to handle multi-step reasoning tasks, few-shot chain-of-thought prompting, and decision-making in multi-agent systems. These agents leverage the impressive planning and reasoning abilities of LLMs to automate complex tasks efficiently. The key characteristics include the ability to navigate through multiple steps of a problem, adapt to various scenarios, and make informed decisions based on the context provided. The benefits of using a plan-and-solve agent in LLM include enhanced problem-solving capabilities, improved efficiency in decision-making processes, and the potential to streamline complex tasks that require reasoning and planning.'

In [7]:
action_chain.run(question)



> Entering new AgentExecutor chain...
I should search for the definition of a plan-and-solve agent in the context of LLM.
Action: Wikipedia
Action Input: Plan-and-solve agent
Observation: Page: Multi-agent planning
Summary: In computer science multi-agent planning involves coordinating the resources and activities of multiple agents.
NASA says, "multiagent planning is concerned with planning by (and for) multiple agents. It can involve agents planning for a common goal, an agent coordinating the plans (plan merging) or planning of others, or agents refining their own plans while negotiating over tasks or resources. The topic also involves how agents can do this in real time while executing plans (distributed continual planning). Multiagent scheduling differs from multiagent planning the same way planning and scheduling differ: in scheduling often the tasks that need to be performed are already decided, and in practice, scheduling tends to focus on algorithms for specific problem doma

'In the context of LLM, a plan-and-solve agent involves coordinating the resources and activities of multiple agents.'